In [30]:
import os
from src.utils import read_config, MySQLAgent
import openai
from llama_index.llms.openai import OpenAI


from llama_index.core import SQLDatabase, Settings
from llama_index.core.query_engine import NLSQLTableQueryEngine, RouterQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector

from sqlalchemy import (
   create_engine,
   MetaData,
   Table,
   Column,
   String,
   Integer,
   select,
   column
)

In [31]:
info_path = '.env/info.json'
info = read_config(info_path)

O_KEY = info['openai']
os.environ["OPENAI_API_KEY"] = O_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

conn_info = info['VM1_news_mysql_conn_info']
sql_agent = MySQLAgent(conn_info)


In [32]:
llm =  OpenAI(temperature=0.1, model="gpt-4o-mini")

In [33]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [34]:
user = conn_info['user']
pw = conn_info['pw']
host = conn_info['host']
port = conn_info['port']
database = conn_info['database']
connection_string = f"mysql+pymysql://{user}:{pw}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(connection_string)

tables = ['cdd_result']

sql_database = SQLDatabase(engine, include_tables=tables)
Settings.llm = llm
Settings.embed_model = embed_model
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables=tables)

In [ ]:
prompt_template_str="""

"""

In [35]:
sql_tool = QueryEngineTool.from_defaults(
   query_engine=sql_query_engine,
   description=('當User詢問Cdd result的時候搜尋這個Table')
)

In [36]:
## RouterQueryEngine 去sql_tool尋找最合適的tool

query_engine =  RouterQueryEngine(
   selector=LLMSingleSelector.from_defaults(),
   query_engine_tools=[sql_tool]
)

In [37]:
query = """旭全發企業有限公司有幾筆資料"""
response = query_engine.query(query)
response.response

'旭全發企業有限公司的資料共有128筆。'

In [45]:
query = """旭全發企業有限公司有幾筆綠燈資料"""
response = query_engine.query(query)
response.response

'旭全發企業有限公司目前沒有任何綠燈資料。'

In [46]:
query = """旭全發企業有限公司有幾筆light_status是green資料"""
response = query_engine.query(query)
response.response

'旭全發企業有限公司有126筆light_status為green的資料。'

In [47]:
response

Response(response='旭全發企業有限公司有126筆light_status為green的資料。', source_nodes=[NodeWithScore(node=TextNode(id_='feb0e716-ca19-4d6c-a08a-9717d9123698', embedding=None, metadata={'sql_query': "SELECT COUNT(*) AS green_count FROM cdd_result WHERE company_name = '旭全發企業有限公司' AND light_status = 'green'", 'result': [(126,)], 'col_keys': ['green_count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, text='[(126,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'feb0e716-ca19-4d6c-a08a-9717d9123698': {'sql_query': "SELECT COUNT(*) AS green_count FROM cdd_result WHERE company_name = '旭全發企業有限公司' AND light_status = 'green'", 'result': [(126,)], 'col_keys': ['green_count']}, 'sql_query': "SELECT COUNT(*) AS green_count FROM cdd_result WHERE company_name = '旭

In [49]:
metadata = response.metadata